In [ ]:

# ==============================================================================
# 0. SETUP & INSTALLATION
# ==============================================================================
import subprocess
import sys
import os

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install dependencies if missing
try:
    import rdkit
    import optuna
    import xgboost
except ImportError:
    print("Installing dependencies... (This may take a minute)")
    install("rdkit")
    install("optuna")
    install("xgboost")

import glob
import re
import time
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import savgol_filter
from scipy.stats import zscore, ttest_rel

# Chemistry
from rdkit import Chem
from rdkit.Chem import Draw

# Machine Learning
import optuna
import joblib
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

# Config
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

# --- GLOBAL CONFIGURATION ---
MODEL_LIST = ['PLS', 'Ridge', 'ElasticNet', 'SVR', 'XGBoost']
N_OPTUNA_TRIALS = 30   # Increased for publication rigor
INNER_CV_FOLDS = 3     # Increased for stability

print(f"--- 🚀 GRAND MASTER PIPELINE (FINAL VERSION) STARTED ---")

# ==============================================================================
# PHASE 1: DATA LOADING
# ==============================================================================
print("\n--- 📂 PHASE 1: Loading Data ---")
base_path = '/content/Raman_Sugars'

if not os.path.exists(base_path):
    print("   > Cloning dataset...")
    os.system(f'git clone https://github.com/Alvaro-FG/Raman_Sugars.git {base_path}')

all_spec = glob.glob(os.path.join(base_path, '**', '*ALL_spectra.csv'), recursive=True)
all_meta = glob.glob(os.path.join(base_path, '**', '*ALL_metadata.csv'), recursive=True)

def load_dataset(spec_f, meta_f):
    df_s = pd.read_csv(spec_f)
    df_m = pd.read_csv(meta_f)
    df_m.columns = [c.split(' [')[0] for c in df_m.columns]

    wc = next(c for c in df_s.columns if 'cm-1' in c or 'Wavenumber' in c)
    wn = pd.to_numeric(df_s[wc].values)
    X = df_s.drop(columns=[wc]).T

    target_cols = ['Sucrose', 'Glucose', 'Fructose', 'Maltose']
    if 'filename' in df_m.columns: df_m = df_m.set_index('filename')

    data = X.join(df_m[target_cols], how='inner')
    X_fin = data.drop(columns=target_cols)
    Y_fin = data[target_cols]

    # Robust Group Parsing
    groups = []
    for f in X_fin.index:
        parts = f.split('_')
        try:
            rc = [p for p in parts if re.match(r'^[A-H]\d+$', p)][0]
            pl = parts[parts.index(rc)+1]
            groups.append(f"{rc}_{pl}")
        except:
            groups.append("Unknown")
    return X_fin, Y_fin, np.array(groups), wn

f_hq_s = next(f for f in all_spec if 'Fast' not in f)
f_hq_m = next(f for f in all_meta if 'Fast' not in f)
X_hq, Y_hq, groups_hq, wavenumbers = load_dataset(f_hq_s, f_hq_m)

# Feature Selection
mask_struct = (wavenumbers >= 800) & (wavenumbers <= 1500)
scenarios = {
    "Structure": X_hq.iloc[:, mask_struct],
    "FullSpec":  X_hq
}

print(f"   > Loaded {X_hq.shape[0]} Samples.")
print(f"   > Scenarios defined: Structure ({mask_struct.sum()} feats) vs FullSpec ({X_hq.shape[1]} feats)")

# ==============================================================================
# PHASE 2: CHEMICAL STRUCTURES
# ==============================================================================
print("\n--- 🧪 PHASE 2: Generating Structures ---")
sugars_smiles = {
    'Glucose': 'C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)O)O)O)O)O',
    'Fructose': 'C([C@@H]1[C@H]([C@@H](C(O1)(CO)O)O)O)O',
    'Sucrose': 'C(O)[C@H]1O[C@H](O[C@]2(CO)O[C@H](CO)[C@@H](O)[C@@H]2O)[C@H](O)[C@@H](O)[C@H]1O',
    'Maltose': 'C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)O[C@@H]2[C@H](O[C@H]([C@@H]([C@H]2O)O)O)CO)O)O)O)O'
}
mols = [Chem.MolFromSmiles(s) for s in sugars_smiles.values()]

img = Draw.MolsToGridImage(mols, molsPerRow=4, subImgSize=(300, 300),
                           legends=list(sugars_smiles.keys()), returnPNG=False)
img.save('molecules.png')
print("   > Molecules saved to molecules.png")

# ==============================================================================
# PHASE 3: MODEL CLASSES & EXECUTION
# ==============================================================================
class SpectralPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, method='none', window=15, deriv=1):
        self.method = method; self.window = window; self.deriv = deriv
    def fit(self, X, y=None): return self
    def transform(self, X):
        X_n = pd.DataFrame(X).copy()
        if self.method == 'snv':
            X_n = X_n.sub(X_n.mean(1), axis=0).div(X_n.std(1), axis=0)
        elif self.method == 'deriv':
            X_n = savgol_filter(X_n, self.window, 2, deriv=self.deriv, axis=1)
        return np.array(X_n)

def run_benchmark(X_data, Y, groups, scenario_name):
    print(f"\n🔬 SCENARIO: {scenario_name} (Features: {X_data.shape[1]})")
    stats = {m: {'R2':[], 'RMSE':[], 'MAE':[], 'Params':None} for m in MODEL_LIST}

    # Store Best Data for Visualization
    best_plot = {'Model': None, 'R2_mean': -999, 'Y_true': None, 'Y_pred': None, 'Pipe': None}

    # Outer Loop: Evaluation
    outer = GroupKFold(n_splits=3)

    for fold, (t_ix, v_ix) in enumerate(outer.split(X_data, Y, groups=groups)):
        print(f"   > Fold {fold+1}...", end=" ")
        X_t, X_v = X_data.iloc[t_ix], X_data.iloc[v_ix]
        y_t, y_v = Y.iloc[t_ix], Y.iloc[v_ix]

        for name in MODEL_LIST:
            # Inner Loop: Optimization
            def obj(trial):
                prep = trial.suggest_categorical('prep', ['none', 'snv', 'deriv'])

                if name=='PLS':
                    # Expanded search space for PLS
                    n_max = min(30, int(X_t.shape[0]*0.8))
                    est = PLSRegression(n_components=trial.suggest_int('n', 2, max(2, n_max)), scale=False)
                elif name=='Ridge':
                    est = Ridge(alpha=trial.suggest_float('a', 0.01, 100, log=True))
                elif name=='ElasticNet':
                    est = ElasticNet(alpha=trial.suggest_float('a', 0.01, 10, log=True),
                                     l1_ratio=trial.suggest_float('l1', 0.1, 0.9))
                elif name=='SVR':
                    est = MultiOutputRegressor(SVR(C=trial.suggest_float('c', 0.1, 100, log=True), epsilon=0.1))
                elif name=='XGBoost':
                    est = MultiOutputRegressor(XGBRegressor(
                        n_estimators=trial.suggest_int('n', 50, 150),
                        max_depth=trial.suggest_int('d', 3, 6),
                        learning_rate=trial.suggest_float('lr', 0.01, 0.2),
                        n_jobs=1, verbosity=0))

                pipe = Pipeline([('p', SpectralPreprocessor(method=prep)),
                                 ('s', StandardScaler()),
                                 ('e', est)])

                # Increased inner folds for stability
                return cross_val_score(pipe, X_t, y_t, cv=INNER_CV_FOLDS, scoring='r2', n_jobs=-1).mean()

            study = optuna.create_study(direction='maximize')
            study.optimize(obj, n_trials=N_OPTUNA_TRIALS)

            # Rebuild Best Pipeline
            bp = study.best_params
            prep = bp.pop('prep')

            if name=='PLS': est = PLSRegression(n_components=bp['n'], scale=False)
            elif name=='Ridge': est = Ridge(alpha=bp['a'])
            elif name=='ElasticNet': est = ElasticNet(alpha=bp['a'], l1_ratio=bp['l1'])
            elif name=='SVR': est = MultiOutputRegressor(SVR(C=bp['c'], epsilon=0.1))
            elif name=='XGBoost': est = MultiOutputRegressor(XGBRegressor(
                n_estimators=bp['n'], max_depth=bp['d'], learning_rate=bp['lr'], n_jobs=-1))

            final = Pipeline([('p', SpectralPreprocessor(method=prep)),
                              ('s', StandardScaler()),
                              ('e', est)])
            final.fit(X_t, y_t)
            p_v = final.predict(X_v)

            stats[name]['R2'].append(r2_score(y_v, p_v, multioutput='uniform_average'))
            stats[name]['RMSE'].append(np.sqrt(mean_squared_error(y_v, p_v)))
            stats[name]['MAE'].append(mean_absolute_error(y_v, p_v))
            stats[name]['Params'] = study.best_params

            curr_r2 = np.mean(stats[name]['R2'])
            if curr_r2 > best_plot['R2_mean']:
                best_plot['R2_mean'] = curr_r2
                best_plot['Model'] = name
                best_plot['Y_true'] = y_v
                best_plot['Y_pred'] = p_v
                best_plot['Pipe'] = final

        print("Done.")
    return stats, best_plot

res_Struct, plots_Struct = run_benchmark(scenarios['Structure'], Y_hq, groups_hq, "Structure")
res_Full, _ = run_benchmark(scenarios['FullSpec'], Y_hq, groups_hq, "FullSpec")

# ==============================================================================
# PHASE 4: PDF REPORT GENERATION
# ==============================================================================
print("\n--- 📄 PHASE 4: Generating Report ---")
pdf_path = os.path.join(base_path, 'Final_Corrected_Report.pdf')

with PdfPages(pdf_path) as pdf:

    # PAGE 1: Intro & Molecules
    fig = plt.figure(figsize=(11, 8.5))
    plt.axis('off')
    plt.text(0.5, 0.95, "CHEMOMETRIC BENCHMARK REPORT", ha='center', fontsize=18, weight='bold')

    if os.path.exists('molecules.png'):
        mol_img = mpimg.imread('molecules.png')
        ax_mol = fig.add_axes([0.1, 0.55, 0.8, 0.35])
        ax_mol.imshow(mol_img); ax_mol.axis('off')

    ax_spec = fig.add_axes([0.1, 0.1, 0.8, 0.4])
    mean_spec = X_hq.mean(0)
    ax_spec.plot(wavenumbers, mean_spec, c='gray', alpha=0.3, label='Rejected Region')
    ax_spec.plot(wavenumbers[mask_struct], mean_spec[mask_struct], c='teal', lw=2, label='Structure Selected')
    ax_spec.set_xlabel("Wavenumber (cm-1)"); ax_spec.set_ylabel("Intensity")
    ax_spec.legend(); ax_spec.set_title("Chemistry-Driven Feature Selection")
    pdf.savefig(fig); plt.close()

    # PAGE 2: Comparison Table
    fig = plt.figure(figsize=(11, 8.5)); plt.axis('off')
    txt = "COMPARATIVE RESULTS (Mean +/- SD)\n"
    txt += f"{'Model':<12} | {'Scenario':<10} | {'R2':<22} | {'RMSE':<22} | {'MAE':<22}\n"
    txt += "="*100 + "\n"

    for m in MODEL_LIST:
        r, rm, ma = res_Struct[m]['R2'], res_Struct[m]['RMSE'], res_Struct[m]['MAE']
        txt += f"{m:<12} | Struct     | {np.mean(r):.4f} ± {np.std(r):.4f} | {np.mean(rm):.2f} ± {np.std(rm):.2f} | {np.mean(ma):.2f} ± {np.std(ma):.2f}\n"
        r, rm, ma = res_Full[m]['R2'], res_Full[m]['RMSE'], res_Full[m]['MAE']
        txt += f"{m:<12} | FullSpec   | {np.mean(r):.4f} ± {np.std(r):.4f} | {np.mean(rm):.2f} ± {np.std(rm):.2f} | {np.mean(ma):.2f} ± {np.std(ma):.2f}\n"
        txt += "-"*100 + "\n"

    plt.text(0.05, 0.95, txt, family='monospace', fontsize=10, va='top')
    pdf.savefig(fig); plt.close()

    # PAGE 3: Diagnostics (Best Model)
    win = plots_Struct['Model']
    y_t = plots_Struct['Y_true']; y_p = plots_Struct['Y_pred']
    targets = ['Sucrose', 'Glucose', 'Fructose', 'Maltose']

    if win is not None:
        fig, axes = plt.subplots(2, 2, figsize=(10, 10))
        fig.suptitle(f"Diagnostics: {win} (Structure-Driven)", fontsize=16)

        for i, ax in enumerate(axes.flat):
            if i < len(targets):
                act = y_t.iloc[:, i].values; pre = y_p[:, i]
                ax.scatter(act, pre, alpha=0.6, c='teal', edgecolor='k')
                mx = max(act.max(), pre.max()); mn = min(act.min(), pre.min())
                ax.plot([mn, mx], [mn, mx], 'k--', lw=2)
                ax.set_title(targets[i])
                ax.set_xlabel("Actual (mg/mL)"); ax.set_ylabel("Predicted (mg/mL)")

                # Inset: Standardized Residuals
                resid = act - pre
                std_val = np.std(resid)
                stud = zscore(resid) if std_val > 1e-9 else np.zeros_like(resid)

                ins = ax.inset_axes([0.6, 0.1, 0.35, 0.35])
                ins.scatter(pre, stud, s=10, c='crimson', alpha=0.5)
                ins.axhline(0, c='k', lw=1)
                ins.axhline(3, c='r', ls=':', lw=1); ins.axhline(-3, c='r', ls=':', lw=1)
                ins.set_title("Std. Resid (Z-score)", fontsize=8) # Corrected Terminology
                ins.set_ylim(-5, 5)
        pdf.savefig(fig); plt.close()

    # PAGE 4: Feature Importance
    pipe = plots_Struct['Pipe']
    imp = None
    if pipe and 'e' in pipe.named_steps:
        est = pipe.named_steps['e']
        try:
            if hasattr(est, 'coef_'): imp = np.abs(est.coef_).mean(axis=0)
            elif hasattr(est, 'estimators_'):
                imps = [e.feature_importances_ for e in est.estimators_]
                imp = np.mean(imps, axis=0)
        except: imp = None

    if imp is not None:
        fig, ax = plt.subplots(figsize=(10, 6))
        sel_wavs = wavenumbers[mask_struct]
        ms = scenarios['Structure'].mean(0).values
        ms_n = (ms - ms.min())/(ms.max()-ms.min())
        ax.plot(sel_wavs, ms_n, c='gray', alpha=0.3, label='Avg Spectrum')
        imp_n = (imp - imp.min())/(imp.max()-imp.min())
        ax.bar(sel_wavs, imp_n, width=2.0, color='crimson', alpha=0.8, label='Importance')
        ax.set_title(f"Feature Importance ({win})"); ax.set_xlabel("Wavenumber (cm-1)"); ax.legend()
        pdf.savefig(fig); plt.close()

    # PAGE 5: Full Hyperparameters (FIXED)
    fig = plt.figure(figsize=(8.5, 11)); plt.axis('off')
    txt = "APPENDIX: OPTIMIZED HYPERPARAMETERS\n\n"
    for m in res_Struct.keys():
        p = res_Struct[m]['Params']
        txt += f"MODEL: {m}\n" + "-"*30 + "\n"
        if p:
            for k, v in p.items(): txt += f"  > {k}: {v}\n"
        txt += "\n"
    plt.text(0.1, 0.9, txt, family='monospace', fontsize=10, va='top')
    pdf.savefig(fig); plt.close()

print(f"✅ FINAL REPORT GENERATED: {pdf_path}")

# Statistical Summary to Console
print("\n--- 📊 STATISTICAL SUMMARY (Structure vs FullSpec) ---")
best_model_name = plots_Struct['Model']
r2_struct = res_Struct[best_model_name]['R2']
r2_full = res_Full[best_model_name]['R2']
print(f"Best Model: {best_model_name}")
print(f"Mean R2 (Structure): {np.mean(r2_struct):.4f}")
print(f"Mean R2 (FullSpec):  {np.mean(r2_full):.4f}")
if len(r2_struct) > 1:
    stat, pval = ttest_rel(r2_struct, r2_full)
    print(f"Paired t-test p-value: {pval:.4f} {'(Significant)' if pval < 0.05 else '(Not Significant)'}")
print("------------------------------------------------------")


--- 🚀 GRAND MASTER PIPELINE (FINAL VERSION) STARTED ---

--- 📂 PHASE 1: Loading Data ---
   > Loaded 1960 Samples.
   > Scenarios defined: Structure (261 feats) vs FullSpec (2000 feats)

--- 🧪 PHASE 2: Generating Structures ---
   > Molecules saved to molecules.png

🔬 SCENARIO: Structure (Features: 261)
   > Fold 1... Done.
   > Fold 2... Done.
   > Fold 3... Done.

🔬 SCENARIO: FullSpec (Features: 2000)
   > Fold 1... Done.
   > Fold 2... Done.
   > Fold 3... Done.

--- 📄 PHASE 4: Generating Report ---
✅ FINAL REPORT GENERATED: /content/Raman_Sugars/Final_Corrected_Report.pdf

--- 📊 STATISTICAL SUMMARY (Structure vs FullSpec) ---
Best Model: Ridge
Mean R2 (Structure): 0.9903
Mean R2 (FullSpec):  0.9805
Paired t-test p-value: 0.1252 (Not Significant)
------------------------------------------------------
